In [ ]:



!pip install python-binance

In [ ]:
!pip install websocket-client

In [ ]:
import websocket, json
import pandas as pd
from binance.client import Client
import datetime as dt
import os

api_key = "ncaGKCQNu9KS8kEwF3Jri37YjsOkKCfLGiKZcq87iThEU3QZIWdxOyeQLUvOCPpG"
api_secret = "tvSQ36Idx7W1QwiMx5ycduhziLQZaHUbhnHZGhqlSRaF6VrkP8QqsLY3LCidthdJ"

client = Client(api_key, api_secret)
symbols = os.listdir('/content/drive/MyDrive/Colab Notebooks/csvs/')
print(symbols)
#symbols = os.listdir('/home/ubuntu/csvs')

['XEMUSDT', 'XRPDOWNUSDT', 'BTGUSDT', 'GNOUSDT', 'OMUSDT', 'ACMUSDT', 'TKOUSDT', 'TORNUSDT', 'GHSTUSDT', 'AIONUSDT', 'BNTUSDT', 'ARDRUSDT', 'VTHOUSDT', 'STEEMUSDT', 'PHAUSDT', 'DGBUSDT', 'LSKUSDT', 'FORTHUSDT', 'RLCUSDT', 'BANDUSDT', 'BNBUPUSDT', 'HIVEUSDT', 'BARUSDT', 'DCRUSDT', 'XVGUSDT', 'QUICKUSDT', 'YFIIUSDT', 'FARMUSDT', 'REPUSDT', 'ALPACAUSDT', 'JOEUSDT', 'VITEUSDT', 'CLVUSDT', 'FIROUSDT', 'PLAUSDT', 'NEARUSDT', 'ROSEUSDT', 'ZILUSDT', 'LTCUSDT', 'DOTUSDT', 'ADAUPUSDT', 'POLSUSDT', 'BURGERUSDT', 'FTMUSDT', 'TUSDUSDT', 'NULSUSDT', 'ICPUSDT', 'KMDUSDT', 'ACHUSDT', 'AVAXUSDT', 'MITHUSDT', 'XECUSDT', 'RUNEUSDT', 'OXTUSDT', 'SHIBUSDT', 'DYDXUSDT', 'PORTOUSDT', 'ACAUSDT', 'MFTUSDT', 'ETCUSDT', 'FRONTUSDT', 'ENSUSDT', 'TFUELUSDT', 'PUNDIXUSDT', 'COTIUSDT', 'AVAUSDT', 'EOSUSDT', 'DOTDOWNUSDT', 'AXSUSDT', 'C98USDT', 'LPTUSDT', 'AUCTIONUSDT', 'TRIBEUSDT', 'SLPUSDT', 'BAKEUSDT', 'EURUSDT', 'MATICUSDT', 'AGLDUSDT', 'CHZUSDT', 'RSRUSDT', 'CTSIUSDT', 'XVSUSDT', 'NEOUSDT', 'GALUSDT', 'FXSUSDT',

In [ ]:
def last_n_min(symbol, lookback:int):
  data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/csvs/"+symbol, names=['E', 'c'])
  #data = pd.read_csv("/home/ubuntu/csvs/"+symbol, names=['E', 'c'])
  data['E'] = pd.to_datetime(data['E'])
  before = pd.Timestamp("now") + dt.timedelta(hours=9) - dt.timedelta(minutes=lookback)
  data = data[data.E >= before]
  return data

prices = last_n_min(symbol, 3)

In [ ]:
rets = []
for symbol in symbols:
  prices = last_n_min(symbol, 3)
  cumret = (prices.c.pct_change() + 1).prod() - 1
  rets.append(cumret)
top_coin = symbols[rets.index(max(rets))]

In [ ]:
print(top_coin)

XEMUSDT


In [ ]:
inv_amt = 200
info = client.get_symbol_info(symbol=top_coin)
Lotsize = float([i for i in info['filters'] if i['filterType'] == 'LOT_SIZE'][0]['minQty'])
prize = float(client.get_symbol_ticker(symbol=top_coin)['price'])
buy_quantity = round(inv_amt/prize/Lotsize)*Lotsize

if float([i for i in client.get_account()['balances'] if i['asset'] == 'USDT'][0]['free']) > 200:
  print('enough funds, order will be executed')
  order = client.order_limit_buy(symbol=top_coin, quantity=buy_quantity, price = prize)
  print(order)
else:
  print('order has not been executed due to already invested')
  quit()
buyprice = float(order['price'])

stream = f"wss://stream.binance.com:9443/ws/{top_coin.lower()}@trade"

def on_message(ws, message):
  msg = json.loads(message)
  if float(msg['p']) < buyprice * 0.98 or float(msg['p']) > 1.025 * buyprice:
    order = client.create_order(symbol=top_coin,
                                side='SELL',
                                type='MARKET',
                                quantity=buy_quantity)
    print(order)
    ws.close()
ws = websocket.WebSocketApp(stream, on_message=on_message)
ws.run_forever()

#crontab -e
#***** /usr/bin/python3 cloud_script.py >> output.txt

{'symbol': 'XEMUSDT', 'status': 'TRADING', 'baseAsset': 'XEM', 'baseAssetPrecision': 8, 'quoteAsset': 'USDT', 'quotePrecision': 8, 'quoteAssetPrecision': 8, 'baseCommissionPrecision': 8, 'quoteCommissionPrecision': 8, 'orderTypes': ['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS_LIMIT', 'TAKE_PROFIT_LIMIT'], 'icebergAllowed': True, 'ocoAllowed': True, 'quoteOrderQtyMarketAllowed': True, 'allowTrailingStop': True, 'isSpotTradingAllowed': True, 'isMarginTradingAllowed': True, 'filters': [{'filterType': 'PRICE_FILTER', 'minPrice': '0.00010000', 'maxPrice': '1000.00000000', 'tickSize': '0.00010000'}, {'filterType': 'PERCENT_PRICE', 'multiplierUp': '5', 'multiplierDown': '0.2', 'avgPriceMins': 5}, {'filterType': 'LOT_SIZE', 'minQty': '1.00000000', 'maxQty': '9222449.00000000', 'stepSize': '1.00000000'}, {'filterType': 'MIN_NOTIONAL', 'minNotional': '10.00000000', 'applyToMarket': True, 'avgPriceMins': 5}, {'filterType': 'ICEBERG_PARTS', 'limit': 10}, {'filterType': 'MARKET_LOT_SIZE', 'minQty'